In [88]:
import numpy as np
import pandas as pd
import requests

from local_configs import enacom_api_key

Tengo que bajar directamente el archivo, porque el API es un poco confuso

In [34]:
# Define the URL with the file to be downlaoded
url = 'https://datosabiertos.enacom.gob.ar/datasets/197452-listado-de-radioaficionados.download/'

# Define the name of the file to be saved locally
filename = 'listado_radioaficionados.xls'

# Send the petition HTTP Get for obtaining the resource
data = requests.get(url)

# Save the file locally
with open(filename, 'wb') as file:
    file.write(data.content)


Con el archivo descargado, convierto la información a un DataFrame

In [98]:
df = pd.read_excel(filename)
df

,licenseeName,callSign,categoryDesc,acto.expirationDate,address.state,address.city
0,MINISTERIO DE SEGURIDAD,LU0CD,SUPERIOR,2025-05-21 00:00:00,Ciudad de Buenos Aires,CABA
1,ARTURO JORGE PEYRU,LU1AAA,NOVICIO,2024-05-07 00:00:00,Ciudad de Buenos Aires,CIUDAD AUTONOMA BUENOS AIRES
2,CHRISTIAN LUIS DIAZ,LU1AAB,NOVICIO,2024-04-26 00:00:00,Ciudad de Buenos Aires,CIUDAD AUTONOMA BUENOS AIRES
3,WENCESLAO BERNARDINO MOREL,LU1AAC,NOVICIO,2024-05-16 00:00:00,Ciudad de Buenos Aires,CIUDAD AUTONOMA BUENOS AIRES
4,JUAN ANTONIO BILOTA,LU1AAD,NOVICIO,2024-08-14 00:00:00,Ciudad de Buenos Aires,CIUDAD AUTONOMA BUENOS AIRES
...,...,...,...,...,...,...
16326,MARIO ALBERTO LEDESMA,LW9HZY,NOVICIO,2026-09-13 00:00:00,Córdoba,LA FALDA
16327,TADEO NICOLAS ACEVEDO,LW9JTA,NOVICIO,2024-02-21 00:00:00,Entre Ríos,COLON
16328,ANGEL ADRIAN ALAIMO FLORES,LW9WAA,NOVICIO,2024-04-30 00:00:00,Chubut,RADA TILLY
16329,ARIEL EDUARDO MONZON,LW9WAM,NOVICIO,2023-12-27 00:00:00,Chubut,PUERTO MADRYN


First, we perform some basic Data Quality operations

In [99]:
# Change dtype of Expiration date colunm
df['acto.expirationDate'] = pd.to_datetime(df['acto.expirationDate'])

# Remove leading or trailing spaces for al object columns
for column in df.columns:
    if df[column].dtype == object:
        df[column] = df[column].str.strip() 

We will check if we have NaNs in the different columns:

In [100]:
df.isna().sum()

licenseeName            0
callSign                0
categoryDesc            0
acto.expirationDate     0
address.state          13
address.city           18
dtype: int64

In [101]:
df['address.state'].unique()

array(['Ciudad de Buenos Aires', 'Provincia de Buenos Aires', nan,
       'Santa Fe', 'Chaco', 'Formosa', 'Córdoba', 'Misiones',
       'Entre Ríos', 'Tucumán', 'Corrientes', 'Mendoza',
       'Santiago del Estero', 'Salta', 'San Juan', 'San Luis',
       'Catamarca', 'La Rioja', 'Jujuy', 'La Pampa', 'Río Negro',
       'Chubut', 'Santa Cruz', 'Tierra del Fuego A. e I.A.S.', 'Neuquén'],
      dtype=object)

We have some instances were the state or the city are not defined. While we can't do anything about the city, we can have a good aproximation of the state by looking at the callsign.

Let's take a look into the missing ones:

In [102]:
nan_state_cond = df['address.state'].isna()

df.loc[nan_state_cond, :]

,licenseeName,callSign,categoryDesc,acto.expirationDate,address.state,address.city
1199,MIGUEL ANGEL BELLOTO,LU1EPW,GENERAL,2023-02-09,NaN,NaN
1587,UNIVERSIDAD NACIONAL DE CORDOBA,LU1HD,SUPERIOR,2023-04-09,NaN,NaN
5433,JUAN CARLOS CHEHADHI,LU3OAB,NOVICIO,2023-10-16,NaN,NaN
5635,RUFINO MAXIMO VELAZQUEZ,LU3XXX,GENERAL,2023-02-09,NaN,NaN
8635,ESTEBAN MANDUCI,LU6EEM,NOVICIO,2023-09-14,NaN,NaN
8757,ESTEBAN RAUL ARISTI,LU6ESQ,NOVICIO,2023-04-13,NaN,NaN
9613,LEONARDO ANTONIO PAGLIARO,LU7DNI,NOVICIO,2023-07-13,NaN,NaN
12639,DIEGO FERNANDO SANCHEZ,LU9FEC,GENERAL,2023-07-18,NaN,NaN
13232,HORACIO MARCELO DE LA ROSA,LU9MZI,NOVICIO,2023-11-05,NaN,NaN
13377,EDUARDO ENRIQUE APABLAZA MILLAR,LU9SAP,GENERAL,2023-02-09,NaN,NaN


We create a function that fills NaN values for address.state based on the callsign structure:

In [103]:
def fix_state_nan(callsign):
    # State is defined by the first letter (and sometimes also second) of the suffix
    first_letter = callsign[3]
    second_letter = callsign[4]

    match first_letter:
        case 'A' | 'B' | 'C':
            return 'Ciudad de Buenos Aires'
        case 'D' | 'E':
            return 'Provincia de Buenos Aires'
        case 'F':
            return 'Santa Fe'
        case 'G':
            if second_letter in [chr(i) for i in range(ord('A'), ord('P'))]:
                return 'Chaco'
            else:
                return 'Formosa'
        case 'H':
            return 'Córdoba'
        case 'I':
            return 'Misiones'
        case 'J':
            return 'Entre Ríos'
        case 'K':
            return 'Tucumán'
        case 'L':
            return 'Corrientes'
        case 'M':
            return 'Mendoza'
        case 'N':
            return 'Santiago del Estero'
        case 'O':
            return 'Salta'
        case 'P':
            return 'San Juan'
        case 'Q':
            return 'San Luis'
        case 'R':
            return 'Catamarca'
        case 'S':
            return 'La Rioja'
        case 'T':
            return 'Jujuy'
        case 'U':
            return 'La Pampa'
        case 'V':
            return 'Río Negro' 
        case 'W':
            return 'Chubut'
        case 'X':
            if second_letter in [chr(i) for i in range(ord('A'), ord('P'))]:
                return 'Santa Cruz'
            else:
                return 'Tierra del Fuego A. e I.A.S.'
        case 'Y':
            return 'Neuquén'
        case 'Z':
            return 'Tierra del Fuego A. e I.A.S.'

    
df['address.state'] = df.apply(lambda row: fix_state_nan(row['callSign']) if pd.isna(row['address.state']) else row['address.state'], axis=1)

Let's check if the fix did it's job:

In [104]:
df.loc[nan_state_cond, :]

,licenseeName,callSign,categoryDesc,acto.expirationDate,address.state,address.city
1199,MIGUEL ANGEL BELLOTO,LU1EPW,GENERAL,2023-02-09,Provincia de Buenos Aires,NaN
1587,UNIVERSIDAD NACIONAL DE CORDOBA,LU1HD,SUPERIOR,2023-04-09,Córdoba,NaN
5433,JUAN CARLOS CHEHADHI,LU3OAB,NOVICIO,2023-10-16,Salta,NaN
5635,RUFINO MAXIMO VELAZQUEZ,LU3XXX,GENERAL,2023-02-09,Tierra del Fuego A. e I.A.S.,NaN
8635,ESTEBAN MANDUCI,LU6EEM,NOVICIO,2023-09-14,Provincia de Buenos Aires,NaN
8757,ESTEBAN RAUL ARISTI,LU6ESQ,NOVICIO,2023-04-13,Provincia de Buenos Aires,NaN
9613,LEONARDO ANTONIO PAGLIARO,LU7DNI,NOVICIO,2023-07-13,Provincia de Buenos Aires,NaN
12639,DIEGO FERNANDO SANCHEZ,LU9FEC,GENERAL,2023-07-18,Santa Fe,NaN
13232,HORACIO MARCELO DE LA ROSA,LU9MZI,NOVICIO,2023-11-05,Mendoza,NaN
13377,EDUARDO ENRIQUE APABLAZA MILLAR,LU9SAP,GENERAL,2023-02-09,La Rioja,NaN
